In [ ]:
# Import packages
import os
import re  # regular expressions
import warnings
import matplotlib.pyplot as plt
import numpy as np
import numpy.ma as ma
import rasterio as rio
from rasterio.plot import plotting_extent
import geopandas as gpd
import earthpy as et
import earthpy.plot as ep
import earthpy.spatial as es
import earthpy.mask as em

import rioxarray as rxr

warnings.simplefilter('ignore')

# Set working directory
os.chdir(os.path.join(et.io.HOME, 'BD', 'BA_DATA'))

In [ ]:
thermal_path = os.path.join("ThermalData_8day", "MOD14A2.A2000049.h08v05.006.2015041131440.hdf")

In [ ]:
vege_path = os.path.join("VegeData_16day", "MOD13A2.A2000049.h08v05.006.2015136104428.hdf")

In [ ]:
tem_path = os.path.join("TemData", "MOD11A2.A2000049.h08v05.006.2015058135046.hdf")

In [ ]:
with rio.open(thermal_path) as dataset:
    print(dataset)
    hdf4_meta = dataset.meta

# Notice that there are metadata at the highest level of the file
hdf4_meta

In [ ]:
# Print all of the subdatasets in the data
with rio.open(thermal_path) as dataset:
    crs = dataset.read_crs()
    for name in dataset.subdatasets:
        print(name)

In [ ]:
# Create empty list to append arrays (of band data)
all_bands = []

# Open the pre-fire HDF4 file
with rio.open(thermal_path) as dataset:
    
    # Loop through each subdataset in HDF4 file
    for name in dataset.subdatasets:
        
        # Use regular expression to identify if subdataset has b0 in the name (the bands)
        if re.search("FireMask", name):
            
            # Open the band subdataset
            with rio.open(name) as subdataset:
                modis_meta = subdataset.profile
                
                # Read band data as a 2 dim arr and append to list
                all_bands.append(subdataset.read(1))

# Stack pre-fire reflectance bands
thermal_modis = np.stack(all_bands)
thermal_modis.shape

In [ ]:
ep.plot_bands(vege_modis,
              scale=False)
plt.show()

In [ ]:
with rio.open(vege_path) as dataset:
    print(dataset)
    hdf4_meta = dataset.meta

# Notice that there are metadata at the highest level of the file
hdf4_meta

In [ ]:
# Print all of the subdatasets in the data
with rio.open(vege_path) as dataset:
    crs = dataset.read_crs()
    for name in dataset.subdatasets:
        print(name)

In [ ]:
# Create empty list to append arrays (of band data)
all_bands = []

# Open the pre-fire HDF4 file
with rio.open(vege_path) as dataset:
    
    # Loop through each subdataset in HDF4 file
    for name in dataset.subdatasets:
        
        # Use regular expression to identify if subdataset has b0 in the name (the bands)
        if re.search("NDVI", name):
            
            # Open the band subdataset
            with rio.open(name) as subdataset:
                modis_meta = subdataset.profile
                
                # Read band data as a 2 dim arr and append to list
                all_bands.append(subdataset.read(1))

# Stack pre-fire reflectance bands
vege_modis = np.stack(all_bands)
vege_modis.shape

In [ ]:
with rio.open(tem_path) as dataset:
    print(dataset)
    hdf4_meta = dataset.meta

# Notice that there are metadata at the highest level of the file
hdf4_meta

In [ ]:
# Print all of the subdatasets in the data
with rio.open(tem_path) as dataset:
    crs = dataset.read_crs()
    for name in dataset.subdatasets:
        print(name)

In [ ]:
# Create empty list to append arrays (of band data)
all_bands = []

# Open the pre-fire HDF4 file
with rio.open(tem_path) as dataset:
    
    # Loop through each subdataset in HDF4 file
    for name in dataset.subdatasets:
        
        # Use regular expression to identify if subdataset has b0 in the name (the bands)
        if re.search("LST_Day", name):
            
            # Open the band subdataset
            with rio.open(name) as subdataset:
                modis_meta = subdataset.profile
                
                # Read band data as a 2 dim arr and append to list
                all_bands.append(subdataset.read(1))

# Stack pre-fire reflectance bands
tem_modis = np.stack(all_bands)
tem_modis.shape

In [ ]:
glr_input = np.stack((thermal_modis[0], vege_modis[0], tem_modis[0]), axis=2)

In [ ]:
glr_input.shape

In [ ]:
glr_input

In [ ]:
glr_input[0][0]

In [ ]:
# change from 3-d array to 2-d array
two_d_input = glr_input.reshape(1440000, 3)

In [ ]:
two_d_input.shape

In [ ]:
two_d_input

In [ ]:
# filter out rows with fill value -3000 for vegetation indices 
a = two_d_input[two_d_input[:,1] != -3000]

In [ ]:
a.shape

In [ ]:
a

In [ ]:
# filter out rows with fill value 0 for surface temperature
b = a[a[:,2] != 0] 

In [ ]:
b.shape

In [ ]:
b

In [ ]:
#filter out rows with value not equal to 5, 7, 8 or 9 for FireMask
c = b[b[:,0] >= 5]
d = c[c[:,0] != 6]

In [ ]:
c.shape

In [ ]:
c

In [ ]:
d.shape

In [ ]:
d

In [ ]:
# combining all the filters
d1 = two_d_input[two_d_input[:,1] != -3000]
d2 = d1[d1[:,2] != 0] 
d3 = d2[d2[:,0] >= 5]
d4 = d3[d3[:,0] != 6]

In [ ]:
d4.shape

In [ ]:
d4

In [ ]:
# set FireMask == 7,8,9 (has fire) to 1, set FireMask == 5 (no fire) to 0
d4[:,0][d4[:,0] >= 7] = 1
d4[:,0][d4[:,0] == 5] = 0

In [ ]:
d4.shape

In [ ]:
d4

In [ ]:
# check if there is fire
max(d4[:,0])